In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import sys
PROJECT_PATH = '/content/drive/My Drive/END/'
sys.path.append(PROJECT_PATH +'/Session7/')
import os
os.chdir(PROJECT_PATH +'/Session7/')

In [ ]:
import pandas as pd

data = pd.read_csv("datasetSentences.csv",encoding = "ISO-8859-1", engine='python')
label = pd.read_csv("sentiment_labels.csv",encoding = "ISO-8859-1", engine='python')
data['labels'] = label[['score']]
data.head()
df = data.copy()
df.head()

,Sentence,labels
0,The Rock is destined to be the 21st Century 's...,2
1,The gorgeously elaborate continuation of `` Th...,2
2,Effective but too-tepid biopic,2
3,If you sometimes like to go to the movies to h...,2
4,"Emerges as something rare , an issue movie tha...",2


In [ ]:
print(df.shape)
df.head()
df.Sentence[0]

(11855, 2)


"The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud  Van Damme or Steven Segal ."

In [ ]:
df.labels.value_counts()

2    6240
1    2175
3    2139
4     663
0     638
Name: labels, dtype: int64

In [ ]:
import random
import torch, torchtext
from torchtext import data

In [ ]:
# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
#df['tran_sentence'] = df.apply(lambda x:translation(x['Sentence']), axis =1)
#df.head()

In [ ]:
Tweet = torchtext.legacy.data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = torchtext.legacy.data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

In [ ]:
type(Tweet)

torchtext.legacy.data.field.Field

In [ ]:
fields = [('tweet', Tweet), ('label', Label)]

In [ ]:
example = [torchtext.legacy.data.Example.fromlist([df.Sentence[i],df.labels[i]], fields) for i in range(df.shape[0])] 

In [ ]:
type(example)
print(*example)

<torchtext.legacy.data.example.Example object at 0x7fbaa705b9d0> <torchtext.legacy.data.example.Example object at 0x7fbaa705b950> <torchtext.legacy.data.example.Example object at 0x7fba636a6a10> <torchtext.legacy.data.example.Example object at 0x7fba636a6850> <torchtext.legacy.data.example.Example object at 0x7fba636a6610> <torchtext.legacy.data.example.Example object at 0x7fbb0dc6a5d0> <torchtext.legacy.data.example.Example object at 0x7fbb0dc6aad0> <torchtext.legacy.data.example.Example object at 0x7fba6272c090> <torchtext.legacy.data.example.Example object at 0x7fba6272c0d0> <torchtext.legacy.data.example.Example object at 0x7fbb0dc6a390> <torchtext.legacy.data.example.Example object at 0x7fba6272c750> <torchtext.legacy.data.example.Example object at 0x7fba6272c8d0> <torchtext.legacy.data.example.Example object at 0x7fba6272ce90> <torchtext.legacy.data.example.Example object at 0x7fba627315d0> <torchtext.legacy.data.example.Example object at 0x7fba62731910> <torchtext.legacy.data.ex

In [ ]:
twitterDataset = torchtext.legacy.data.Dataset(example, fields)

In [ ]:
(train, valid) = twitterDataset.split(split_ratio=[70, 30], random_state = random.seed(SEED))

In [ ]:
len(train), len(valid)

(8298, 3557)

In [ ]:
print(type(valid))
print((vars(valid.examples[1]))['tweet'])
#' '.join(filtered_sentence).split(', ')
for i in range(1,10):
  print(' '.join(vars(valid.examples[i])['tweet']).split(', ')[0])

<class 'torchtext.legacy.data.dataset.Dataset'>
['The', 'art', 'demands', 'live', 'viewing', '.', ' ']
The art demands live viewing .  
An impressive hybrid .  
The film 
` Film aficionados can not help but love Cinema Paradiso 
... there are enough moments of heartbreaking honesty to keep one glued to the screen .  
Starts out with tremendous promise 
The verdict : Two bodies and hardly a laugh between them .  
If you can push on through the slow spots 
Funny and also heartwarming without stooping to gooeyness .  


In [ ]:
Tweet.build_vocab(train)
Label.build_vocab(train)

In [ ]:
print('Size of input vocab : ', len(Tweet.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Tweet.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  17795
Size of label vocab :  5
Top 10 words appreared repeatedly : [(' ', 8298), ('.', 7831), (',', 7004), ('the', 5906), ('and', 4332), ('a', 4292), ('of', 4280), ('to', 2968), ('-', 2644), ("'s", 2512)]
Labels :  defaultdict(None, {2: 0, 1: 1, 3: 2, 4: 3, 0: 4})


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
train_iterator, valid_iterator = torchtext.legacy.data.BucketIterator.splits((train, valid), batch_size = 32, 
                                                            sort_key = lambda x: len(x.tweet),
                                                            sort_within_batch=True, device = device)

In [ ]:
next(iter(train_iterator))



[torchtext.legacy.data.batch.Batch of size 32]
	[.tweet]:('[torch.cuda.LongTensor of size 32x10 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.LongTensor of size 32 (GPU 0)]

In [ ]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Tweet.vocab.stoi, tokens)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)   
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

In [ ]:
# Define hyperparameters
size_of_vocab = len(Tweet.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = 5
num_layers = 2
dropout = 0.2

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [ ]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(17795, 300)
  (encoder): LSTM(300, 100, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=100, out_features=5, bias=True)
)
The model has 5,580,605 trainable parameters


In [ ]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        tweet, tweet_lengths = batch.tweet  
        
        # convert to 1D tensor
        predictions = model(tweet, tweet_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.label)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.label)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            tweet, tweet_lengths = batch.tweet
            
            # convert to 1d tensor
            predictions = model(tweet, tweet_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

	Train Loss: 1.407 | Train Acc: 51.79%
	 Val. Loss: 1.389 |  Val. Acc: 51.76% 

	Train Loss: 1.378 | Train Acc: 52.82%
	 Val. Loss: 1.387 |  Val. Acc: 51.94% 

	Train Loss: 1.377 | Train Acc: 52.80%
	 Val. Loss: 1.386 |  Val. Acc: 51.94% 

	Train Loss: 1.376 | Train Acc: 52.88%
	 Val. Loss: 1.387 |  Val. Acc: 51.94% 

	Train Loss: 1.376 | Train Acc: 52.88%
	 Val. Loss: 1.387 |  Val. Acc: 51.94% 

	Train Loss: 1.377 | Train Acc: 52.79%
	 Val. Loss: 1.388 |  Val. Acc: 51.94% 

	Train Loss: 1.376 | Train Acc: 52.95%
	 Val. Loss: 1.389 |  Val. Acc: 51.88% 

	Train Loss: 1.373 | Train Acc: 53.10%
	 Val. Loss: 1.385 |  Val. Acc: 51.94% 

	Train Loss: 1.367 | Train Acc: 53.72%
	 Val. Loss: 1.395 |  Val. Acc: 51.06% 

	Train Loss: 1.357 | Train Acc: 54.96%
	 Val. Loss: 1.401 |  Val. Acc: 50.23% 



In [ ]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_tweet(tweet):
    
    categories = {0: "very Negative", 1:"Negative", 2:"Neutral", 3: "positive", 4: "very positive"}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(tweet)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [ ]:
classify_tweet("A valid explanation for why Trump won't let women on the golf course.")

'very Negative'

In [ ]:
print("Tweet", "                     ","prediction", "         ", "Actual")

categories = {0: "very Negative", 1:"Negative", 2:"Neutral", 3: "positive", 4: "very positive"}
count = 0
for i in range(1,1000):
  text = ' '.join(vars(valid.examples[i])['tweet']).split(', ')[0]
  if (len (text) > 10) and (classify_tweet(text) == categories [vars(valid.examples[i])['label']]):
    print (text,"  ", classify_tweet(text), "       ", categories [vars(valid.examples[i])['label']] )
    print("\"")
    count= count +1 
    if count >= 10:
      break

    


Tweet                       prediction           Actual
The verdict : Two bodies and hardly a laugh between them .      very Negative         very Negative
"
The first mistake     Negative         Negative
"
Any movie that makes hard work seem heroic deserves a look .      very Negative         very Negative
"
But is that knot from dramatic tension or a symptom of artistic malnutrition ?      very Negative         very Negative
"
What starts off as a potentially incredibly twisting mystery becomes simply a monster chase film .      very Negative         very Negative
"
Spectacularly beautiful     Neutral         Neutral
"
With very little to add beyond the dark visions already relayed by superb recent predecessors like Swimming With Sharks and The Player     Neutral         Neutral
"
Its direction     Neutral         Neutral
"
The movie is as padded as Allen 's jelly belly .      very Negative         very Negative
"
The trashy teen - sleaze equivalent of Showgirls .      very Negative